In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow.compat.v1 as tf
import tensorflow_hub as hub
import contractions
from bs4 import BeautifulSoup
import unicodedata
import re

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install tensorflow-hub
!pip install contractions
!pip install beautifulsoup4

In [ ]:
dataset = pd.read_csv("/kaggle/input/quora-insincere-questions-classification/train.csv")

In [ ]:
x = dataset['question_text'].values
y = dataset['target'].values

train_x = x[:30000]
train_y = y[:30000]

val_x = x[30000:35000]
val_y = y[30000:35000]

test_x = x[35000:]
test_y = y[35000:]
train_x.shape, val_x.shape, test_x.shape

In [ ]:
def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    [s.extract() for s in soup(['iframe', 'script'])]
    stripped_text = soup.get_text()
    stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
    return stripped_text


def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text


def expand_contractions(text):
    return contractions.fix(text)



def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', text)
    return text


def pre_process_document(document):
    
    document = strip_html_tags(document)
    document = document.lower()
    document = document.translate(document.maketrans("\n\t\r", "   ")) 
    document = remove_accented_chars(document)   
    document = expand_contractions(document)   
    special_char_pattern = re.compile(r'([{.(-)!}])')
    document = special_char_pattern.sub(" \\1 ", document)
    document = remove_special_characters(document, remove_digits=True)  
    document = re.sub(' +', ' ', document)
    document = document.strip()
    
    return document


pre_process_corpus = np.vectorize(pre_process_document)

In [ ]:
train_x = pre_process_corpus(train_x)
val_x = pre_process_corpus(val_x)
test_x = pre_process_corpus(test_x)

In [ ]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {'sentence': train_x}, train_y, 
    batch_size=256, num_epochs=None, shuffle=True)

predict_train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {'sentence': train_x}, train_y, shuffle=False)

predict_val_input_fn = tf.estimator.inputs.numpy_input_fn(
    {'sentence': val_x}, val_y, shuffle=False)

predict_test_input_fn = tf.estimator.inputs.numpy_input_fn(
    {'sentence': test_x}, test_y, shuffle=False)

In [ ]:
embedding_feature = hub.text_embedding_column(
    key='sentence', 
    module_spec="https://tfhub.dev/google/universal-sentence-encoder/2",
    trainable=False)

In [ ]:
dnn = tf.estimator.DNNClassifier(
          hidden_units=[512, 128],
          feature_columns=[embedding_feature],
          n_classes=2,
          activation_fn=tf.nn.relu,
          dropout=0.1,
          optimizer=tf.train.AdagradOptimizer(learning_rate=0.005))    

In [ ]:
tf.logging.set_verbosity(tf.logging.ERROR)

TOTAL_STEPS = 1000
STEP_SIZE = 100
for step in range(0, TOTAL_STEPS+1, STEP_SIZE):
    print()
    print('-'*100)
    print('Training for step =', step)
    dnn.train(input_fn=train_input_fn, steps=STEP_SIZE)
    print('Eval Metrics (Train):', dnn.evaluate(input_fn=predict_train_input_fn))
    print('Eval Metrics (Validation):', dnn.evaluate(input_fn=predict_val_input_fn))